In [23]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
from sklearn.metrics import f1_score

### Set up device

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [25]:
data_dir = '../data'
train_dir = 'train_data'
test_dir = 'test_data_v2'
val_dir = 'val_data'

In [26]:
image_exts = ['jpg', 'jpeg', 'bmp', 'png']

### load data

In [27]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [28]:
train_data = datasets.ImageFolder(root=os.path.join(data_dir, train_dir), transform=transform)
val_data = datasets.ImageFolder(root=os.path.join(data_dir, val_dir), transform=transform)

In [29]:
len(val_data)

15990

In [30]:
train_loader = DataLoader(train_data, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)

### Build CNN model

In [31]:
import torch.nn as nn
import torch.nn.functional as F

In [32]:
class BinaryCNN(nn.Module):
    def __init__(self):
        super(BinaryCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64* 32 *32, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        

In [33]:
model = BinaryCNN().to(device)

In [34]:
model.parameters

<bound method Module.parameters of BinaryCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=65536, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)>

### Train the model

In [ ]:
import torch.optim


criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(lr=0.0001, params=model.parameters())

num_epoches = 2

for epoch in range(num_epoches):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device, dtype=torch.float32)
        
        labels = labels.view(-1, 1)
        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch: {epoch + 1}, Loss: {running_loss/len(train_loader)}")
torch.save(model.state_dict(), '../model/base_CNN_1.pth')

Epoch: 1, Loss: 0.345093435049057
Epoch: 2, Loss: 0.2795462098121643


### Evaluate|

In [36]:
model.load_state_dict(torch.load('../model/base_CNN_1.pth'))

<All keys matched successfully>

In [37]:
model.eval()
prediction_list = []
labels_list = []
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.view(-1, 1).float()
        outputs = model(images)
        predicted = (outputs > 0.5).float()
        predicted = predicted.squeeze().to(torch.int).tolist()
        labels = labels.squeeze().to(torch.int).tolist()
        prediction_list.extend(predicted)
        labels_list.extend(labels)

In [38]:
print('F1-score: ', f1_score(y_true=labels_list, y_pred=prediction_list))

F1-score:  0.8775523777851679


### Predict

In [39]:
test_df = pd.read_csv('../data/test.csv')

In [40]:
image_list = list(test_df['id'])

In [41]:
outputs_list = []
for image_path in image_list:
    image = Image.open(os.path.join(data_dir, image_path)).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0).to(device)
    with torch.no_grad():
        logit = model(image)
    prob = torch.sigmoid(logit)
    prediction = (prob>0.5).float()
    outputs_list.append(int(prediction.item()))

In [42]:
test_df['label'] = outputs_list

In [43]:
test_df.to_csv('../data/result/CNN_pred.csv', index=False)